In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

2024-11-19 18:31:13.510705: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-19 18:31:13.618784: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-19 18:31:13.701918: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732037473.785448   30695 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732037473.807961   30695 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-19 18:31:13.989013: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

# Préparation du jeu de données

In [2]:
import splitfolders
data_dir = 'dataset_Multiclass_Covid'

In [3]:
# Diviser les données en données train et test
splitfolders.ratio(data_dir, output='dataset', seed=555, ratio=(.8, .2))

Copying files: 532 files [00:00, 949.75 files/s] 


In [4]:
# générer les images : faible pourcentage d'images du dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_directory('dataset/train', target_size=(64, 64), batch_size=32, class_mode='categorical')

Found 424 images belonging to 4 classes.


In [6]:
test_generator = test_datagen.flow_from_directory('dataset/val', target_size=(64, 64), batch_size=32, class_mode='categorical')

Found 108 images belonging to 4 classes.


# Contruction du DEEP CNN

In [7]:
# initialisation du CNN
classifier = Sequential()

In [8]:
# première couche de convolution
classifier.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 3)))
classifier.add(MaxPooling2D(pool_size=(2, 2)))


/home/ariel/PycharmProjects/enronmails/venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
W0000 00:00:1732037489.147464   30695 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [9]:
# deuxième couche de convolution
classifier.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Dropout(0.2))

In [10]:
# troisième couche de convolution
classifier.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

In [11]:
# vectorisation
classifier.add(Flatten())
classifier.add(Dropout(0.3))

In [12]:
# ANN
classifier.add(Dense(512, activation='relu'))
classifier.add(Dense(128, activation='relu'))
classifier.add(Dense(4, activation='softmax'))

In [13]:
classifier.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,519,236 (9.61 MB)

 Trainable params: 2,519,236 (9.61 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:
# création d'un model checkpoint
checkpointer = ModelCheckpoint(filepath='checkpoint/classifier.keras', monitor='accuracy', mode='max', verbose=1, save_best_only=True)

In [16]:
# implémentation pour l'arret precoce
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min')

In [17]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

In [18]:
callbacks_list = [checkpointer, early_stopping, reduce_lr]

In [19]:
classifier.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,519,236 (9.61 MB)

 Trainable params: 2,519,236 (9.61 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
model = classifier.fit(train_generator, validation_data = test_generator, epochs = 20, verbose = 1, callbacks=callbacks_list)

/home/ariel/PycharmProjects/enronmails/venv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
 1/14 ━━━━━━━━━━━━━━━━━━━━ 24s 2s/step - accuracy: 0.2500 - loss: 1.3963

2024-11-19 18:31:46.641671: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25189632 exceeds 10% of free system memory.
2024-11-19 18:31:46.642042: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25189632 exceeds 10% of free system memory.


 2/14 ━━━━━━━━━━━━━━━━━━━━ 4s 348ms/step - accuracy: 0.2266 - loss: 1.4363

2024-11-19 18:31:47.001459: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25189632 exceeds 10% of free system memory.
2024-11-19 18:31:47.004373: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25189632 exceeds 10% of free system memory.


 3/14 ━━━━━━━━━━━━━━━━━━━━ 3s 349ms/step - accuracy: 0.2205 - loss: 1.4438

2024-11-19 18:31:47.358667: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 25189632 exceeds 10% of free system memory.


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 334ms/step - accuracy: 0.2323 - loss: 1.4420
Epoch 1: accuracy improved from -inf to 0.25000, saving model to checkpoint/classifier.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 8s 484ms/step - accuracy: 0.2335 - loss: 1.4405 - val_accuracy: 0.2500 - val_loss: 1.3830 - learning_rate: 0.0010
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 317ms/step - accuracy: 0.3098 - loss: 1.3782
Epoch 2: accuracy improved from 0.25000 to 0.34434, saving model to checkpoint/classifier.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 410ms/step - accuracy: 0.3121 - loss: 1.3783 - val_accuracy: 0.4259 - val_loss: 1.3618 - learning_rate: 0.0010
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 324ms/step - accuracy: 0.4012 - loss: 1.3429
Epoch 3: accuracy improved from 0.34434 to 0.39151, saving model to checkpoint/classifier.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 412ms/step - accuracy: 0.4005 - loss: 1.3404 - val_accuracy: 0.5926 - val_loss: 1.0925 - learning_rate: 0.0010
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 321m

In [21]:
train_generator.class_indices

{'Infection_Bacterienne': 0,
 'Infection_Covid': 1,
 'Infection_Virale': 2,
 'Normal': 3}

In [22]:
from sklearn.metrics import classification_report
y_pred_train = classifier.predict(test_generator)
y_predict = np.argmax(y_pred_train, axis=1)
print(classification_report(test_generator.classes, y_predict, target_names=test_generator.classes_))

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 271ms/step


AttributeError: 'DirectoryIterator' object has no attribute 'classes_'